In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.options.display.max_rows = 999
import geopandas as gpd

# df = pd.read_csv('sintetico_circus_income_YQ'+T+'.csv')
df = pd.read_csv('./data/sintetico_circus_income_YQ_deflac.csv', usecols = ['RADIO_REF_ID', 'V01', 'DPTO', 'circuito', 'H05', 'H06', 'H07', 'H08',
       'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP',
       'IX_TOT', 'AGLOMERADO', 'PERSONA_REF_ID', 'P02', 'P03', 'P05', 'P07',
       'P08', 'P09', 'P10', 'CONDACT', 'dpto_merge', 'CAT_OCUP', 'CAT_INAC',
       'CH07', 'P47T'])

df['P47T'] = df['P47T']*1420 # precios de Sep 2018

df['circuito'] = df['circuito'].astype(str).str.zfill(5)#.astype(str) # Aparente bug por el cual los str quedan como numeros y depues no matchea el circuito


FileNotFoundError: File b'./data/sintetico_circus_income_YQ_deflac.csv' does not exist

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
df.columns

### Lo mas facil seria modificar df, de forma que los dptos pasen a llamarse por su seccion y sus circuitos pasen a llamarse por el partido. 

In [3]:
# Acomodamos los dptos segun cantidad de electores, creamos los df de referencia con estas nuevas areas

secciones = pd.read_csv('./data/secciones.csv')

ordered = df.groupby('dpto_merge')['PERSONA_REF_ID'].count().sort_values(ascending = False).reset_index()

n_partidos = 65

dptos_agrupados = ordered.iloc[n_partidos:].merge(secciones)[['label', 'dpto_merge']]
dptos_agrupados['area_0'] = dptos_agrupados['label']; dptos_agrupados = dptos_agrupados.drop('label', 1)
dptos_agrupados['area_1'] = dptos_agrupados['dpto_merge']

dptos_sueltos = ordered.iloc[:n_partidos].merge(df[['dpto_merge', 'circuito']].drop_duplicates())[['dpto_merge', 'circuito']]
dptos_sueltos['area_0'] = dptos_sueltos['dpto_merge']
dptos_sueltos['area_1'] = dptos_sueltos['circuito']

ordered['grouped'] = (ordered.index >= n_partidos).astype(int)
ordered = ordered.drop('PERSONA_REF_ID', 1)

NameError: name 'df' is not defined

In [18]:
# Se lo ponemos al df. Tenemos el result

df = df.merge(ordered)

df_suelto = df.loc[df.grouped == 0].merge(dptos_sueltos)
df_agrup = df.loc[df.grouped == 1].merge(dptos_agrupados)

df_final = pd.concat([df_suelto, df_agrup])


## Primero lo primero. El total de electores por lugar

In [19]:
df_final.shape
# sampleo randomizado de un numero igual al 10% de los electores (1.527.885). No hace falta cargar 15 millones de personas, mas liviano multiplicar el numero al final. 
# La precision es de +- 10

(1527885, 36)

In [20]:
# area_0 / area_1 representan Partido/circuito o bien zona_gde/partidos (cuando los partidos son chicos se agrupan)
electoresxlugar = 10*df_final.groupby(['area_0','area_1']).count().rename(columns = {'PERSONA_REF_ID': 'n_electores'})[['n_electores']]
electoresxlugar.to_csv('./n_electores.csv')
electoresxlugar.sort_values(by = 'n_electores', ascending = False).head(20).sort_index()

n_electores
area_0             area_1             
almirante brown    00019         96650
florencio varela   00302        117520
general pueyrredon 00364        135510
                   00365        119500
                   00366         88460
                   00367         72540
general rodriguez  00374         75290
jose c. paz        00398         83820
                   0398A         75170
la matanza         00635        115900
                   0635A        104590
                   0635B        103040
lomas de zamora    00583        206750
                   00587         94740
merlo              0652A        139020
moreno             0663C         83960
moron              00665        104920
pilar              00768         77400
tandil             00933        104120
tres de febrero    00391         82800

In [21]:
result_comb = df_final.groupby(['area_0', 'area_1'])['P47T'].describe()

#Refinado
result_comb = result_comb.round(-1).clip(0)[['25%', '50%', '75%', 'count']].rename(columns = {'count': 'electores', '50%':'mediana'}).astype(int)
name_l0 = 'Ingresos (basado en EPH)'
result_comb = pd.concat([result_comb], axis=1, keys=[name_l0])
result_comb[(name_l0, 'electores')] = result_comb[(name_l0, 'electores')]*10

result_comb.to_csv('./result_comb_circ.csv')

In [22]:
result_comb = 10*df_final.groupby(['area_0', 'CONDACT', 'CAT_OCUP', 'CAT_INAC'])[['PERSONA_REF_ID']].count()
# dg.sort_values(by = 'PERSONA_REF_ID',ascending = False).cumsum().iloc[400:].diff()#/15278850.
dg = result_comb
dg.index.set_levels(['', 'Jubilado', 'Rentista', 'Estudiante', 'Ama de casa', 'Menor', 'Discapacitado', 'Otros'], level = 'CAT_INAC', inplace = True)
dg.index.set_levels(['', 'Patron', 'Cta Propia', 'Obrero', 'Familiar', 'Otros'], level = 'CAT_OCUP', inplace = True)
dg.index.set_levels(['OCUPADO', 'DESOCUP', 'INACTIVO'], level = 'CONDACT', inplace = True)
table = dg.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).loc[result_comb.PERSONA_REF_ID > 1000].unstack(0)['PERSONA_REF_ID'].T

table.to_csv('./cat_ocups.csv')

In [23]:
# Hacemos lo mismo con el de votos

In [24]:
votos = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_cnt.csv', index_col=[0, 1, 2], header = [0, 1])
votos.index = votos.index.droplevel(-1)
ordered_mi = pd.concat([ordered.set_index('dpto_merge')], axis=1, keys=['key'])
pd.concat([votos, ordered_mi], axis = 1)

votos_ = pd.concat([votos], axis=1, keys=['VOTOS'])
votos_
votos_list = votos_.stack(-2, -1).stack().reset_index()
votos_list = votos_list.merge(ordered, left_on = ['dpto_merger_str'], right_on = ['dpto_merge'])
votos_list['circuito'] = votos_list['circuito'].str.zfill(5)

votos_list_suelto = votos_list.loc[votos_list.grouped == 0].merge(dptos_sueltos, how = 'left')
votos_list_suelto['area_0'] = votos_list_suelto['dpto_merge']; votos_list_suelto['area_1'] = votos_list_suelto['circuito'] # parche porque algunos circuitos se pierden en el merge

votos_list_agrup = votos_list.loc[votos_list.grouped == 1].merge(dptos_agrupados, how = 'left')

votos_list_comb = pd.concat([votos_list_suelto, votos_list_agrup])

# #Listo, ahora lo vamos a agrupar y darle la forma del dataset de votos original

votos_comb = votos_list_comb.groupby(['area_0', 'area_1', 'level_2', 'level_3'])[['VOTOS']].sum()

votos_comb = votos_comb.unstack(-2, -1).unstack()['VOTOS'].reindex(votos.columns, axis = 1)

In [25]:
votos_comb.to_csv('./data/votos_comb.csv')

In [26]:
# Votos los quiero en pct y por area

votos_area = votos_comb.groupby(level = 0).sum()
r_list = []
for eleccion in ['DN_2015', 'paso_2017', 'DN_2017']:
    votos_area_eleccion = votos_area.T.loc[votos_area.columns.get_level_values(1) == eleccion].T
    votos_area_eleccion_pct = 100*votos_area_eleccion.div(votos_area_eleccion.iloc[:, 1], 0)
    r_list += [votos_area_eleccion_pct]
    
votos_area_pct = pd.concat(r_list,1)
votos_area_pct = votos_area_pct.reindex(votos.columns, axis = 1)
votos_area_pct.to_csv('./votos_area_pct.csv')

# Combine GeoDataFrames

In [27]:
gdf = gpd.read_file('./../EleccionesARG/data/circuitos/clean/clean.shp')
gdf['circuito'] = gdf['circuito'].str.zfill(5)
# print(len(gdf))
gdf = gdf.merge(ordered)
# print(len(gdf))

gdf_suelto = gdf.loc[gdf.grouped == 0].merge(dptos_sueltos, how = 'left')
gdf_suelto['area_0'] = gdf_suelto['dpto_merge']; gdf_suelto['area_1'] = gdf_suelto['circuito'] # parche porque algunos circuitos se pierden en el merge
gdf_agrup = gdf.loc[gdf.grouped == 1].merge(dptos_agrupados)

gdf_suelto = gdf_suelto.dissolve(by = ['area_0', 'area_1'])[['PROV_REF_I','provincia','dpto_merge','geometry']]
gdf_agrup = gdf_agrup.dissolve(by = ['area_0', 'area_1'])[['PROV_REF_I','provincia','dpto_merge','geometry']]
# print(len(gdf_suelto) + len(gdf_agrup))

gdf = gpd.GeoDataFrame(pd.concat([gdf_suelto, gdf_agrup]).reset_index())
gdf.to_file('./../EleccionesARG/data/circuitos/custom/BA_custom_areas.shp')

# Listo

In [28]:
votos = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_cnt.csv', index_col=[0, 1, 2], header = [0, 1])
votos.head()

ELECTORES                   VOTANTES  \
                                          DN_2015 paso_2017 DN_2017  DN_2015   
dpto_merger_str    circuito PROV_REF_ID                                        
lomas de zamora    0583     2              169408    168438  206753   128955   
merlo              0652A    2              124717    133454  139025    99983   
general pueyrredon 0364     2              131245    130613  135506    96119   
                   0365     2              116626    118106  119503    85912   
florencio varela   0302     2              105141    111845  117519    77521   

                                                          CAMBIEMOS            \
                                        paso_2017 DN_2017   DN_2015 paso_2017   
dpto_merger_str    circuito PROV_REF_ID                                         
lomas de zamora    0583     2              113817  146146     20154     20376   
merlo              0652A    2               97533  106765     16500     18555   
general pueyrredon 0364     2               88235   97139     31925     33111   
                   0365     2               80400   85772     27464     28953   
florencio varela   0302     2               79155   85593     18141     21999   

                                                 \
                                        DN_2017   
dpto_merger_str    circuito PROV_REF_ID           
lomas de zamora    0583     2             30975   
merlo              0652A    2             26215   
general pueyrredon 0364     2             42261   
                   0365     2             36660   
florencio varela   0302     2             27853   

                                        ALIANZA FRENTE PARA LA VICTORIA  \
                                                                DN_2015   
dpto_merger_str    circuito PROV_REF_ID                                   
lomas de zamora    0583     2                                     75191   
merlo              0652A    2                                     48031   
general pueyrredon 0364     2                                     30847   
                   0365     2                                     29359   
florencio varela   0302     2                                     32231   

                                                               ...                         \
                                                               ...                          
dpto_merger_str    circuito PROV_REF_ID                        ...                          
lomas de zamora    0583     2                                  ...                          
merlo              0652A    2                                  ...                          
general pueyrredon 0364     2                                  ...                          
                   0365     2                                  ...                          
florencio varela   0302     2                                  ...                          

                                            1PAIS         VOTOS BLANCOS  \
                                        paso_2017 DN_2017       DN_2015   
dpto_merger_str    circuito PROV_REF_ID                                   
lomas de zamora    0583     2               13950   11433         11271   
merlo              0652A    2               15267   11849          8624   
general pueyrredon 0364     2               12547   10738          9377   
                   0365     2               11245    9344          8890   
florencio varela   0302     2                9260    7384          5526   

                                                           \
                                        paso_2017 DN_2017   
dpto_merger_str    circuito PROV_REF_ID                     
lomas de zamora    0583     2               10765    5049   
merlo              0652A    2                4826    3203   
general pueyrredon 0364     2                6450    3834   
                   0365     

In [29]:
votos_comb

ELECTORES            \
                                                     DN_2015 paso_2017   
area_0                area_1                                             
Resto 1a Seccion      general la madrid               9049.0    9482.0   
                      general las heras              11780.0   12219.0   
                      navarro                        13342.0   13935.0   
                      suipacha                        9053.0    8889.0   
Resto 2a Seccion      capitan sarmiento              11654.0   12195.0   
                      carmen de areco                12471.0   11994.0   
                      rojas                          19766.0   20012.0   
                      san andres de giles            18564.0   19057.0   
                      san antonio de areco           19532.0   18376.0   
Resto 3a Seccion      ayacucho                       15595.0   17082.0   
                      magdalena                      13631.0   14843.0   
                      punta indio                     8723.0    9521.0   
Resto 4a Seccion      alberti                         8859.0    9300.0   
                      carlos casares                 18383.0   18971.0   
                      carlos tejedor                  9841.0    9487.0   
                      florentino ameghino             7572.0    7986.0   
                      general arenales               14241.0   14222.0   
                      general pinto                   9187.0    9354.0   
                      general viamonte               14230.0   14980.0   
                      hipolito yrigoyen               7173.0    8009.0   
                      leandro n. alem                14647.0   14745.0   
                      rivadavia                      13233.0   13976.0   
Resto 5a Seccion      castelli                        7007.0    7559.0   
                      general belgrano               15449.0   15223.0   
                      general guido                   2457.0    2461.0   
                      general juan madariaga         16354.0   17352.0   
                      general lavalle                 3422.0    4098.0   
                      general paz                     9625.0   10227.0   
                      loberia                        14239.0   14195.0   
                      maipu                           9500.0    9887.0   
                      monte                          17951.0   18670.0   
                      pila                            3011.0    3164.0   
                      rauch                          12274.0   13227.0   
                      san cayetano                    7082.0    6499.0   
                      tordillo                        1734.0    1521.0   
Resto 6a Seccion      adolfo alsina                  13189.0   13985.0   
                      adolfo gonzales chaves         10420.0   10473.0   
                      benito juarez                  16447.0   16959.0   
                      coronel de marina l. rosales   47359.0   48824.0   
                      coronel dorrego                13617.0   13765.0   
                      coronel pringles               19687.0   20282.0   
                      daireaux                       14225.0   14485.0   
                      guamini                         8839.0    9833.0   
                      laprida                         9156.0    9376.0   
                      monte hermoso                   7909.0    7807.0   
                      pellegrini                      5149.0    5136.0   
                      puan                           14299.0   14433.0   
                      saavedra                       17678.0   18272.0   
                      salliquelo                      7363.0    7768.0   
                      tornquist                      11422.0   12945.0   
                      tres lomas                      6405.0    6236.0   
Resto 7a Seccion      general alvear                  7

### Combine result, nivel dptos, resto seccion

In [ ]:
# result = df.groupby(['DPTO', 'dpto_merge', 'circuito', 'ANO4', 'TRIMESTRE', 'CONDACT', 'CAT_INAC'])['P47T'].describe()


# # result = result.loc[result['count'] > 1000]
# # result.index.set_levels(['', 'Jubilado', 'Rentista', 'Estudiante', 'Ama de casa', 'Menor', 'Discapacitado', 'Otros'], level = 'CAT_INAC', inplace = True)
# # result.index.set_levels(['OCUPADO', 'DESOCUP', 'INACTIVO'], level = 'CONDACT', inplace = True)

# # nos quedamos con los grandes
# large_100 = result.sort_values(by = 'count',ascending = False).reset_index()['dpto_merge'][:n_partidos].values
# result = result.loc[large_100]

# #agrupamos los chicos en secciones. Repetimos la agregacion
# secciones = pd.read_csv('./secciones.csv')

# df_rest = df.loc[~df.dpto_merge.isin(large_100)] # 549100 electores
# df_rest = df_rest.merge(secciones[['dpto_merge', 'label']])
# # df_rest.loc[df_rest.seccion == 1].groupby(['dpto_merge', 'circuito']).count()['RADIO_REF_ID'] #para chequear como quedaron los 'resto de secciones'

# result_sec = df_rest.groupby(['label'])['P47T'].describe()
# result_sec.index.name = result.index.name

# result_comb = pd.concat([result, result_sec]).sort_values(by = '50%')
# result_comb['count'] = result_comb['count'].astype(int)
# # result_comb.index = result_comb.index.droplevel(0)


# votos = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_dpto_cnt.csv', index_col=[0, 1], header = [0, 1])
# votos.index = votos.index.droplevel(1)
# votos_main = votos.loc[large_100]
# votos_rest = votos.reset_index().loc[~votos.reset_index().dpto_merger_str.isin(large_100)]# 549100 electores

# votos_rest = votos_rest.merge(secciones[['dpto_merge', 'label']], left_on = 'dpto_merger_str', right_on = 'dpto_merge')
# votos_rest = votos_rest.groupby(['label']).sum(); votos_rest.columns = votos.columns


# votos_comb = pd.concat([votos_main, votos_rest])

# votos_comb.to_csv('./votos_comb.csv')
# result_comb.to_csv('./result_comb.csv')



### Combine results nivel circuito, partidos en resto seccion

In [ ]:

# result = df.groupby(['DPTO', 'dpto_merge', 'circuito', 'ANO4', 'TRIMESTRE', 'CONDACT', 'CAT_INAC'])['P47T'].describe()

result = df.groupby(['dpto_merge'])['P47T'].describe()

# result = result.loc[result['count'] > 1000]
# result.index.set_levels(['', 'Jubilado', 'Rentista', 'Estudiante', 'Ama de casa', 'Menor', 'Discapacitado', 'Otros'], level = 'CAT_INAC', inplace = True)
# result.index.set_levels(['OCUPADO', 'DESOCUP', 'INACTIVO'], level = 'CONDACT', inplace = True)

# nos quedamos con los grandes
large_100 = result.sort_values(by = 'count',ascending = False).reset_index()['dpto_merge'][:n_partidos].values
result = result.loc[large_100]

#agrupamos los chicos en secciones. Repetimos la agregacion
secciones = pd.read_csv('./secciones.csv')

df_rest = df.loc[~df.dpto_merge.isin(large_100)] # 549100 electores
df_rest = df_rest.merge(secciones[['dpto_merge', 'label']])
# df_rest.loc[df_rest.seccion == 1].groupby(['dpto_merge', 'circuito']).count()['RADIO_REF_ID'] #para chequear como quedaron los 'resto de secciones'

result_sec = df_rest.groupby(['label'])['P47T'].describe()
result_sec.index.name = result.index.name


result_comb = pd.concat([result, result_sec]).sort_values(by = '50%')
result_comb['count'] = result_comb['count'].astype(int)
# result_comb.index = result_comb.index.droplevel(0)
result_comb = pd.concat([result_comb], axis=1, keys=['Ingresos segun EPH'])
result_comb[('Ingresos segun EPH', 'count')] = result_comb[('Ingresos segun EPH', 'count')]*10


votos = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_dpto_cnt.csv', index_col=[0, 1], header = [0, 1])
votos.index = votos.index.droplevel(1)
votos_main = votos.loc[large_100]
votos_rest = votos.reset_index().loc[~votos.reset_index().dpto_merger_str.isin(large_100)]# 549100 electores

votos_rest = votos_rest.merge(secciones[['dpto_merge', 'label']], left_on = 'dpto_merger_str', right_on = 'dpto_merge')
votos_rest = votos_rest.groupby(['label']).sum(); votos_rest.columns = votos.columns


votos_comb = pd.concat([votos_main, votos_rest])

votos_comb.to_csv('./votos_comb.csv')
result_comb.to_csv('./result_comb.csv')

In [ ]:
pd.DataFrame(list(set(votos.index.values) - set(large_100)), columns = ['dpto_merge']).merge(secciones).sort_values(by = ['label', 'dpto_merge'])[['label', 'dpto_merge']]

In [ ]:
pd.concat([result_comb,votos_comb], axis = 1)


## Concentracion de electores

In [ ]:
sorted_ = votos_comb.iloc[:, 2].sort_values(ascending = False)
orden = (100 * sorted_.cumsum()/sorted_.sum()).round(1)
orden.head()

In [ ]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
s = result.copy()
s = np.round(result_dpto[['count','25%','50%','75%']], -1).sort_values(by = '50%')
s.style.background_gradient(cmap=cm)


## Salario medio por circuito

In [ ]:
import geopandas as gpd

circuito_shp = gpd.read_file('./../EleccionesARG/data/circuitos/clean/clean.shp')
circuito_shp['circuito'] = circuito_shp['circuito'].str.zfill(5)
circuito_shp = circuito_shp.dissolve(by = ['dpto_merge','circuito'])

quantile = '50%'
income = result[[quantile]]
# income.index = income.index.droplevel([0, 3, 4])
income.index = income.index.droplevel([0])

gdf = gpd.GeoDataFrame(pd.concat([income, circuito_shp], axis = 1))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8, 10))

fig, axs = plt.subplots(1, 2, figsize = (15, 12))
# lst = list(activity)
# lst[0] = 'RCA'
# activity = tuple(lst)

for i in range(2):
    ax = axs[i]
    gdf.plot(column = quantile, vmin = 8000, vmax = 16000, cmap = 'RdYlGn', linewidth = 0, ax = ax)

    if i == 0: ax.set(xlim = (-64, -56), ylim = (-41.4, -33))#; ax.set_title(str(activity)+'\nProvincia Buenos Aires')
    if i == 1: ax.set(xlim = (-59.0, -57.9), ylim = (-35.2, -33.9))#; ax.set_title(str(activity)+'\nGran Buenos Aires')

    ax.axis('off')
plt.show()

## Salario medio vs votos

In [ ]:
## Traer data de ingresos

In [ ]:
result_comb = pd.read_csv('./result_comb_circ.csv', header = [0, 1], index_col = [0, 1])

data = result_comb.reset_index()

# data = data.loc[(data.CONDACT == 'OCUPADO') & (data.CAT_INAC == '')& (data['count'] > 1000)]
data = data.loc[(data.iloc[:, -1] > 20000)]#.drop(['CONDACT', 'CAT_INAC'], axis = 1)
# data.sample(50).sort_values(by = '50%')

In [ ]:
data = data.set_index(['area_0', 'area_1'])
data.index.names = ['area_0', 'area_1']

# data = pd.concat([data], axis=1, keys=['Ingresos']); data.columns = data.columns.swaplevel()
# data = data['Ingresos']
# data.columns = data.columns.swaplevel()

data.head()

In [ ]:
15344720/(1.527890e+06)

In [ ]:

# votos = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_cnt.csv', index_col=[0, 1, 2], header = [0])
votos = pd.read_csv('./votos_comb.csv', index_col=[0, 1], header = [0])
# Toda esta vuelta es para ponerle nombres familiares a las listas. Pandas no deja setear nombres repetidos en nivel de multiindex
votos.columns = ['ELECTORES', 'ELECTORES', 'ELECTORES', 'VOTANTES', 'VOTANTES',
       'VOTANTES', 'CAMBIEMOS', 'CAMBIEMOS', 'CAMBIEMOS', 'FPV / UC', 'FPV / UC', 'FPV / UC', 'Massa', 'Massa', 'Massa', 'BLANCOS', 'BLANCOS', 'BLANCOS',
       'FJ (Randazzo)', 'FJ (Randazzo)', 'FIT', 'FIT', 'FIT']
cols = list(zip(votos.columns, votos.iloc[0,:]))
c1 = pd.MultiIndex.from_tuples(cols, names=['Fuerza', 'Eleccion'])
votos = pd.DataFrame(data=votos[2:].values, columns=c1, index=votos.index[2:])
votos.index.names = ['area_0','area_1']
votos = votos.astype(float).astype(int)
votos.head()

In [ ]:
votos = pd.read_csv('./votos_area_pct.csv', index_col=[0], header = [0])

votos.columns = ['ELECTORES', 'ELECTORES', 'ELECTORES', 'VOTANTES', 'VOTANTES',
       'VOTANTES', 'CAMBIEMOS', 'CAMBIEMOS', 'CAMBIEMOS', 'FPV / UC', 'FPV / UC', 'FPV / UC', 'Massa', 'Massa', 'Massa', 'BLANCOS', 'BLANCOS', 'BLANCOS',
       'FJ (Randazzo)', 'FJ (Randazzo)', 'FIT', 'FIT', 'FIT']
cols = list(zip(votos.columns, votos.iloc[0,:]))
c1 = pd.MultiIndex.from_tuples(cols, names=['Fuerza', 'Eleccion'])
votos = pd.DataFrame(data=votos[2:].values, columns=c1, index=votos.index[2:])
votos.index.names = ['area_0']
votos = votos.astype(float).round(1)#.astype(int)

votos.sample(3).sample(3, axis = 1)


In [ ]:
# Fue Reemplazada por votos_comb. La unica diferencia es ['dpto_merger_str', 'circuito']..

# df = pd.read_csv('./../../Documents/EleccionesARG/data/tablas_votos/VOTOS_2015_17_circ_pct.csv', index_col=[0, 1, 2], header = [0, 1])
# # df['UNIDAD CIUDADANA'] = df['UNIDAD CIUDADANA'] + df['FRENTE JUSTICIALISTA'] 
# # df = df.drop('FRENTE JUSTICIALISTA', axis = 1)
# # df = df.diff(2, axis = 1).iloc[:, df.columns.get_level_values(1) == 'DN_2017'].

# # Arreglar circuitos
# df.index = df.index.set_levels(df.index.levels[1].str.zfill(5), level=1)
# df.index = df.index.droplevel(2)

# df.head()

In [ ]:
# pal = sns.color_palette("tab10")
# pal.as_hex()

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
pd.options.display.max_columns = 99

quantile = 'mediana'

###

d = pd.concat([data, votos], axis = 1).dropna().reset_index(drop = True)
# d = d[[( 'Ingresos', 'electores'),(quantile, 'Ingresos'), ('UNIDAD CIUDADANA', 'DN_2017'), ('CAMBIEMOS', 'DN_2017'), ('ALIANZA FRENTE PARA LA VICTORIA', 'DN_2015'), ('CAMBIEMOS', 'DN_2015')]]
d.columns = d.columns.map('_'.join)
d['size'] = np.sqrt(d['Ingresos (basado en EPH)_electores'])

###

# ['Ingresos (basado en EPH)_25%', 'Ingresos (basado en EPH)_mediana',
#        'Ingresos (basado en EPH)_75%', 'Ingresos (basado en EPH)_electores',
#        'ELECTORES_DN_2015', 'ELECTORES_paso_2017', 'ELECTORES_DN_2017',
#        'VOTANTES_DN_2015', 'VOTANTES_paso_2017', 'VOTANTES_DN_2017',
#        'CAMBIEMOS_DN_2015', 'CAMBIEMOS_paso_2017', 'CAMBIEMOS_DN_2017',
#        'FPV / UC_DN_2015', 'FPV / UC_paso_2017', 'FPV / UC_DN_2017',
#        'Massa_DN_2015', 'Massa_paso_2017', 'Massa_DN_2017', 'BLANCOS_DN_2015',
#        'BLANCOS_paso_2017', 'BLANCOS_DN_2017', 'FJ (Randazzo)_paso_2017',
#        'FJ (Randazzo)_DN_2017', 'FIT_paso_2017', 'FIT_DN_2017', 'FIT_DN_2015'],
#       dtype='object')
    
    

fig, axs = plt.subplots(2, 2, figsize = (16, 12))

ax = axs[0, 0]
plot_UC17 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y = 'FPV / UC_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .8,'s': d['size']}, color = '#1f77b4', label = 'UC 17')
plot_PRO17 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='CAMBIEMOS_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .8, 's': d['size']}, color = '#ff7f0e', label = 'CAMB 17')
ax.legend()
ax.set_title('UC vs CAMBIEMOS en 2017', fontsize = 16)
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 65))


###
ax = axs[0, 1]
plot_UC15 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='FPV / UC_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .5, 's': d['size']}, color = '#1f77b4', label = 'FPV 15')
plot_PRO15 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='CAMBIEMOS_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .5, 's': d['size']}, color = '#ff7f0e', label = 'CAMB 15')
ax.legend()
ax.set_title('FPV vs CAMBIEMOS en 2015', fontsize = 16)
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 65))


###
ax = axs[1, 0]
plot_UC17 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='FPV / UC_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .8, 's': d['size']}, color = '#1f77b4', label = 'UC 17')
plot_UC15 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='FPV / UC_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .5, 's': d['size']}, color = '#1f77b4', label = 'FPV 15')
ax.legend()
ax.set_title('FPV 15 / UC 17', fontsize = 16)
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 65))


###
ax = axs[1, 1]
plot_PRO17 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='CAMBIEMOS_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .8,'s': d['size']}, color = '#ff7f0e', label = 'CAMB 17')
plot_PRO15 = sns.regplot(x= 'Ingresos (basado en EPH)_'+quantile, y='CAMBIEMOS_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .5, 's': d['size']}, color = '#ff7f0e', label = 'CAMB 15')
ax.legend()
ax.set_title('CAMBIEMOS mejoro del 15 al 17', fontsize = 16)
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 65))



# pd.concat([data, votos], axis = 1).dropna().plot(x=('Ingresos', '50%'), y=('FPV / UC', 'DN_2017'), marker = '.', linewidth = 0, ax = axs)
# ax.xscale('log')
# axs.set_yscale('log')

In [ ]:
d

In [ ]:
print('\n'.join(d.columns))

In [ ]:
pd.options.display.max_columns = 99

quantile = '50%'

###

d = pd.concat([data, df], axis = 1).dropna().reset_index(drop = True)
# d = d[[('count', 'Ingresos'),(quantile, 'Ingresos'), ('UNIDAD CIUDADANA', 'DN_2017'), ('1PAIS', 'DN_2017'), ('ALIANZA FRENTE PARA LA VICTORIA', 'DN_2015'), ('ALIANZA UNIDOS POR UNA NUEVA ALTERNATIVA (UNA)', 'DN_2015')]]
d.columns = d.columns.map('_'.join)
d['size'] = np.sqrt(d['count_Ingresos'])

###

fig, axs = plt.subplots(2, 2, figsize = (16, 12))

# d['1PAIS_DN_2017'] = d['1PAIS_DN_2017']*4
# d['FRENTE JUSTICIALISTA_DN_2017'] = d['FRENTE JUSTICIALISTA_DN_2017']*6

ax = axs[0, 0]
sns.regplot(x= quantile+'_Ingresos', y = 'UNIDAD CIUDADANA_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, label = 'UC 17')
sns.regplot(x= quantile+'_Ingresos', y='1PAIS_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, color = '#9467bd', label = '1PAIS 17')
sns.regplot(x= quantile+'_Ingresos', y='FRENTE JUSTICIALISTA_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, color = '#8c564b', label = 'FJ 17')
ax.set_title('UC vs PJ y Massa en 2017', fontsize = 16)
ax.legend()
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 60))


###

ax = axs[0, 1]
sns.regplot(x= quantile+'_Ingresos', y='ALIANZA FRENTE PARA LA VICTORIA_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .6, 's': d['size']}, color = '#1f77b4', label = 'FPV 15')
sns.regplot(x= quantile+'_Ingresos', y='ALIANZA UNIDOS POR UNA NUEVA ALTERNATIVA (UNA)_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .7, 's': d['size']}, color = '#9467bd', label = 'Massa 15')
ax.set_title('FPV vs Massa en 2015', fontsize = 16)
ax.legend()
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 60))


###

ax = axs[1, 0]
sns.regplot(x= quantile+'_Ingresos', y='ALIANZA FRENTE PARA LA VICTORIA_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .6, 's': d['size']}, color = '#1f77b4', label = 'FPV 15')
sns.regplot(x= quantile+'_Ingresos', y='UNIDAD CIUDADANA_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, color = '#1f77b4', label = 'UC 17')
sns.regplot(x= quantile+'_Ingresos', y='FRENTE JUSTICIALISTA_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, color = '#8c564b', label = 'FJ 17')
ax.set_title('FPV a UC + FJ en 2017', fontsize = 16)
ax.legend()
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 60))

###

ax = axs[1, 1]
sns.regplot(x= quantile+'_Ingresos', y='ALIANZA UNIDOS POR UNA NUEVA ALTERNATIVA (UNA)_DN_2015', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .7, 's': d['size']}, color = '#9467bd', label = 'Massa 15')
sns.regplot(x= quantile+'_Ingresos', y='1PAIS_DN_2017', data = d, marker = '.', ax = ax, scatter_kws = {'alpha': .9,'s': d['size']}, color = '#9467bd', label = '1Pais 17')
ax.set_title('Esp. Massa cosecho menos en 2017', fontsize = 16)
ax.legend()
ax.set(xlabel = 'INGRESO MEDIANO (pesos corr Sep-18)', ylabel = 'PORCENTAJE VOTOS', ylim = (0, 60))


# pd.concat([data, df], axis = 1).dropna().plot(x=('Ingresos', '50%'), y=('UNIDAD CIUDADANA', 'DN_2017'), marker = '.', linewidth = 0, ax = axs)
# ax.xscale('log')
# axs.set_yscale('log')

In [ ]:
'Analisis de EPH en circuito.ipynb'